In [26]:
# Utils
import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import os
import torchvision.models as models
from utility.early_stopping import EarlyStopping
# Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import torch.optim as optim
from torch.utils.data import DataLoader, SubsetRandomSampler


import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split
from tensorboardX import SummaryWriter

# sklearn
from sklearn.metrics import confusion_matrix, classification_report

**Seed Setting**

In [16]:
random.seed(0)

**Device Setting**

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

**Setting Hyperparameter**

In [18]:
batch_size = 128
num_epochs = 20
learning_rate = 0.001
momentum = 0.9

# **Data PreProcessing**

In [21]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    # 무작위로 -15도에서 +15도 사이 회전
    transforms.RandomRotation(degrees=15),
    # 밝기, 대비, 채도, 색조 변형 (임의 값)
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761])
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761])
])

In [24]:
train_data = datasets.CIFAR100(root='./data', train=True, download=True, transform=train_transform)
test_data = datasets.CIFAR100(root='./data', train=False, download=True, transform=test_transform)

Files already downloaded and verified
Files already downloaded and verified


**Split Train Data**

In [30]:
total_train = len(train_data)
# train_data의 10%로 val_data 분할
val_size = int(0.1 * total_train)
train_size = total_train - val_size

train_data, val_data = random_split(train_data, [train_size, val_size])

In [31]:
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2)